In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()
uploaded_2 = files.upload()
import io

Saving Train.csv to Train.csv


Saving Test.csv to Test.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['Train.csv']), sep=",")
df_test = pd.read_csv(io.BytesIO(uploaded_2['Test.csv']), sep=",")

<h2>Data Preparation</h2>


<h3>Treating Na Values</h3>

In [ ]:
#getting features with string and numerical datatype
feat_object = list(i for i in df.columns if df[i].dtype.name == "object")
feat_num = list(i for i in df.columns if df[i].dtype.name != "object")

In [ ]:
#getting object features with nan values
feat_object_na = [i for i in feat_object if (df[i].count()/df.shape[0]) != 1]

In [ ]:
for i in feat_object_na:
  df[i].fillna(df[i].mode()[0], inplace=True)

feat_object_na = [i for i in feat_object if (df_test[i].count()/df_test.shape[0]) != 1]
feat_object_na
for i in feat_object_na:
  df_test[i].fillna(df_test[i].mode()[0], inplace=True)

df.shape, df_test.shape,df.columns, df_test.columns

((12079, 14),
 (5177, 13),
 Index(['ID', 'Policy Start Date', 'Policy End Date', 'Gender', 'Age',
        'First Transaction Date', 'No_Pol', 'Car_Category',
        'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State',
        'ProductName', 'target'],
       dtype='object'),
 Index(['ID', 'Policy Start Date', 'Policy End Date', 'Gender', 'Age',
        'First Transaction Date', 'No_Pol', 'Car_Category',
        'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State',
        'ProductName'],
       dtype='object'))

In [ ]:
train_df = df.copy().drop('target', True)
target_series = df.copy()["target"]
test_df = df_test.copy()
train_data_len = target_series.shape[0]
train_df.shape, test_df.shape, target_series.shape,train_data_len

((12079, 13), (5177, 13), (12079,), 12079)

In [ ]:
merged_df = pd.concat([train_df, test_df], axis=0)
merged_df.shape, merged_df.head()

((17256, 13),         ID Policy Start Date  ...  State  ProductName
 0  0040R73        2010-05-14  ...  Lagos  Car Classic
 1  0046BNK        2010-11-29  ...  Lagos  Car Classic
 2  005QMC3        2010-03-21  ...  Lagos  Car Classic
 3  0079OHW        2010-08-21  ...  Lagos      CarSafe
 4  00BRP63        2010-08-29  ...  Lagos        Muuve
 
 [5 rows x 13 columns])

In [ ]:
merged_df["Policy Start Date"] = pd.to_datetime(merged_df["Policy Start Date"])
merged_df["Policy End Date"] = pd.to_datetime(merged_df["Policy End Date"])
merged_df["First Transaction Date"] = pd.to_datetime(merged_df["First Transaction Date"])


In [ ]:
dt_columns = ["Policy Start Date","Policy End Date","First Transaction Date"]
attr = ["year", "month", "day"]
for col in dt_columns:
  for att in attr:
    merged_df[f"{col} {att.title()}"] = getattr(merged_df[col].dt, att)

In [ ]:
to_be_excluded_feat = ["ID","Policy Start Date","Policy End Date","First Transaction Date",'LGA_Name']
working_df = pd.get_dummies(merged_df.drop(to_be_excluded_feat, axis=1), drop_first=True)

In [ ]:
X_train_df = working_df.iloc[:train_data_len, :]
X_test_df = working_df.iloc[train_data_len:, :]
X_test_df.shape, X_train_df.shape

((5177, 271), (12079, 271))

In [ ]:
X = X_train_df.copy()
y = target_series.copy()


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_test_df = scaler.transform(X_test_df)

In [ ]:
seed = 42

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

In [ ]:
# #XGBOOST
# from sklearn.ensemble import RandomForestClassifier
# params = {
#     'max_depth':4,
#     'n_estimators':1000,
#     'class_weight':"balanced_subsample"
# }
# model_1 = RandomForestClassifier(random_state=seed, **params)
# model_1.fit(X_train, y_train)
# predictions = model_1.predict(X_test)
# print(classification_report(y_test, predictions))
# print(confusion_matrix(y_test, predictions))
# print(f1_score(y_test, predictions))

In [ ]:
#XGBOOST
from lightgbm import LGBMClassifier
params = {
    'max_depth':5,
    'min_child_weight':1,
    'gamma':0,
    'subsample':0.8,
    'colsample_bytree':0.8,
    'scale_pos_weight':1.0,
    'learning_rate':0.1,
    'n_estimators':1000,
    'class_weight':"balanced"
}
model_2 = LGBMClassifier(random_state=24, **params)
model_2.fit(X_train, y_train)
predictions = model_2.predict(X_test)
print(num)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(f1_score(y_test, predictions))

In [ ]:
# #XGBOOST
# from xgboost import XGBClassifier
# params = {
#     'max_depth':5,
#     'min_child_weight':1,
#     'gamma':0,
#     'subsample':0.8,
#     'colsample_bytree':0.8,
#     'scale_pos_weight':1.0,
#     'learning_rate':0.1,
#     'n_estimators':1000,
#     'class_weight':'balanced'
# }
# model_3 = XGBClassifier(random_state=seed, **params)
# model_3.fit(X_train, y_train)
# predictions = model_3.predict(X_test)
# print(classification_report(y_test, predictions))
# print(confusion_matrix(y_test, predictions))
# print(f1_score(y_test, predictions))

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import make_scorer,f1_score
# param_ = {
#     'max_depth': range(3,10,2),
#     'min_child_weight':range(1,6,2)

# }
# params = {
#     'max_depth':5,
#     'min_child_weight':1,
#     'gamma':0,
#     'subsample':0.8,
#     'colsample_bytree':0.8,
#     'scale_pos_weight':1,
#     'learning_rate':0.1,
#     'n_estimators':1000,
#     'class_weight':"balanced"
# }
# model = LGBMClassifier(random_state=seed, **params)
# grid_search = GridSearchCV(estimator=model,param_grid=param_, scoring=make_scorer(f1_score), n_jobs=-1,iid=False,cv=5)
# grid_search.fit(X_train,y_train)


In [ ]:
#getting object features with nan values


In [ ]:
test_id = df_test["ID"]
test_predictions = model_2.predict(X_test_df)

In [ ]:

test_id.shape, test_predictions.shape

In [ ]:
ser_submimssion = pd.Series(data=test_predictions, index=test_id)
ser_submimssion.name = "target"
ser_submimssion

In [ ]:
a = ser_submimssion.to_csv("Submission.csv")

In [ ]:
a

In [ ]:
# from google.colab import drive
# drive.mount('drive')

In [ ]:
# ser_submimssion.to_csv("Submission3.csv")

In [ ]:
# !cp Submission3.csv "drive/My Drive"

In [ ]:
model_2.feature_importances_

In [ ]:
feat_imp = model.feature_importances_

In [ ]:
feat_imp_label = working_df.columns
feat_imp_label, feat_imp_label.shape

In [ ]:
ser_feat_imp = pd.Series(dict(zip(feat_imp_label, feat_imp)))

In [ ]:
ser_feat_imp = ser_feat_imp.sort_values(ascending=False)

In [ ]:
ser_feat_imp

In [ ]:
feat_important_label = ser_feat_imp.index.values

In [ ]:
feat_important_label